## Step 01 - Data Cleaning
The above mentioned low resolution data needs some preparation before processing. The data set is full of null values, real data can be found at only few points. For this reason we need to clean it. First delete all rows with no data points at all. (If we have only NaN values in a row, then we drop that row.) We save the result to "CleandFromAllNull.xlsx".

After clearing the all-NaN rows we interpolate the rest. If we have 2 points with value and NaN-s between, we assume steady consumption there. On the other hand the input data set has aggregated data (how much electricity has been used until the actual time point) but we need electricity useage in every half-hour time interval (not aggregated).
Eg.

|day1 |day2 |day3|day4|day5 |day6|... |
| --- | --- | - | --- | --- | - |---  |
|NaN  |NaN  |**3**|NaN  |NaN  |**6**|...  |

Here we start with NaN-s, so until the first value we change all NaN-s to 0-s. After the first value (**3**) we have 2 more NaN-s and then the second value (**6**). Now we assume steady consumption between our **3** and **6**. The difference between the values is **6-3** = 3, this 3 value is for 4 days (day3, day4, day5, day6) so the portion of each time-interval is 3/4 = 0.75.
The above example data will be processed like:

|day1 |day2 |day3|day4|day5 |day6|... |
| --- | --- | -- | -- | --- | -- |---  |
|0  |0  |**3.75**  |0.75  |0.75 |**0.75**|...  |

Now we have our test data ready.

In [1]:
%pylab inline
import pandas as pd
import datetime
import ipywidgets
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
xls = pd.ExcelFile('MPAN Data - interim 080819.xlsx')

In [3]:
'''
The original input excel file had more sheets.
Create a dict with sheet names as keys and sheets as values.
'''
sheet_to_df_map = {}
for sheet in xls.sheet_names:
    sheet_to_df_map[sheet] = xls.parse(sheet)

In [189]:
# in this case we only need one sheet

df = sheet_to_df_map['2018 2019']
df.columns = df.iloc[0]   # fixing header
df = df.drop(df.index[0])
df.head()
df.shape

(2909, 589)

In [5]:
'''
counting from 0 the 9th column is the first
with reading data, lets find this with a 
script:
'''
for i in range(len(df.columns.tolist())):
    if type(df.columns.tolist()[i]) == datetime.datetime:
        fromHere = i
        break
print(fromHere) # prints "9",  automatic finding OK.
shpe = df.shape

print(shpe[0], shpe[1])

pb = ipywidgets.IntProgress(max = shpe[0] - 1)
display(pb)

# Delete empty rows

indicesOfEmptyRows = []
index = 0
for j in range(shpe[0]):
    if df.iloc[j,fromHere:].isnull().values.all():
        indicesOfEmptyRows.append(j)
    pb.value = index
    index += 1
df2 = df.drop(indicesOfEmptyRows, inplace=False)
df2.to_excel("CleandFromAllNull.xlsx")

9
2909 589


A Jupyter Widget

In [123]:
# save data cleand from empty rows to excel file

df2 = pd.read_excel("CleandFromAllNull.xlsx")
df = df2.reset_index(drop=True)

In [190]:
indicesWhereNotEnoughDataPoints = []  # if there is only 1 data point in a whole row -> delete it
pb = ipywidgets.IntProgress(max = df.shape[0] - 1)
display(pb)
ind = 0

for i in range(df.shape[0]):
    pb.value = ind
    ind += 1
    whereIsData = (np.where(df.iloc[i,fromHere:].isnull().values == False))[0] # returns array of indices of real datapoints
    data = df.iloc[i,fromHere:].values

    if len(whereIsData) < 2:
        indicesWhereNotEnoughDataPoints.append(i)
    else:
        begin = whereIsData[0]
        beginHasFisrtValue = True
        if begin > 0:
            for k in range(begin):
                data[k] = 0
        for j in range(1, len(whereIsData)):

            end = whereIsData[j]
            
            value = (data[end]-data[begin]) / (end-begin) # daily portion of electricity usage
            
            #  if the 'value' is negative -> means wrong data -> put 0-s to these days as value
            if value < 0:
                for k in range(begin, end):
                    data[k] = 0
            else:
                if beginHasFisrtValue:
                    data[begin] = value
                    beginHasFisrtValue = False
                else:
                    data[begin] = (data[begin] + value) / 2
                for k in range(begin+1, end+1):
                    data[k] = value
            begin = end
        for j in range(end, len(data)):
            data[j] = 0
            
    df = changeValues(df, data, i, arange(fromHere, df.shape[1]))
df2 = df.drop(indicesWhereNotEnoughDataPoints, inplace = False)

A Jupyter Widget

In [196]:
df2.to_csv("Interpolated.csv")

In [19]:
numberOfDataPoints = len(df.iloc[i,fromHere:])-sum(df.iloc[0,fromHere:].isnull().values)

579

In [33]:
whereIsData = np.where(df.iloc[0,fromHere:].isnull().values == False) # returns array of indices of real datapoints

In [167]:
df.head()

,First Read,Last Read,Advance,First Read,Last Read,Days,Annual Advance,Readings,MPAN ID,2018-01-01 00:00:00,...,2019-07-25 00:00:00,2019-07-26 00:00:00,2019-07-27 00:00:00,2019-07-28 00:00:00,2019-07-29 00:00:00,2019-07-30 00:00:00,2019-07-31 00:00:00,2019-08-01 00:00:00,2019-08-02 00:00:00,2019-08-03 00:00:00
1,1752911,1752911,0,00:00:00,00:00:00,0,,1,6,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1375558,1657284,281726,00:00:00,00:00:00,00:00:00,,19,9,1375558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,0,00:00:00,00:00:00,0,,0,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1136382,1223559,87177,00:00:00,00:00:00,0,,18,13,1136382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0,0,0,00:00:00,00:00:00,0,,0,2806,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
whereIsData = (np.where(df.iloc[1,fromHere:].isnull().values == False))[0]
df.iloc[1, 8:10]

0
MPAN ID                      9
2018-01-01 00:00:00    1375558
Name: 2, dtype: object

In [114]:
def changeValues(df, mylist, row, indices):
    '''
    @summary: changes a set of values within DataFrame's row to values from parameter mylist
    @param df: DataFrame in which we make the changes
    @type df: pandas.core.frame.DataFrame
    @param mylist: list with values to change DataFrame's elements
    @type mylist: list
    @param row: the index of row in which we make the changes
    @type row: int
    @param indices: the number indices of columns in which we make changes
    @type indices: list
    '''
    columnNames = df.columns
    if len(mylist) != len(indices):
        print("Error! len(mylist) != len(indices)")
        return 0
    for i in range(len(indices)):
        df.loc[row, columnNames[indices[i]]] = mylist[i]
    return df